<a href="https://colab.research.google.com/github/Ilya-D-Sleptsov/programming/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B8%20%D0%BF%D0%BE%20%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0%203%20%22%D0%A5%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Слепцов Илья
# Задания практического занятия 3 "Холодильник"

In [8]:
from datetime import date
from decimal import Decimal

# Глобальный словарь с продуктами
goods = {}

# Словарь для хранения единиц измерения по названию продукта
units = {}


def add(name, amount, expiration_date=None, unit=None):
    """
    Добавляет продукт в холодильник
    unit — единица измерения: 'кг', 'шт', 'л' и т.д. (опционально)
    """
    amount = Decimal(str(amount))

    if name not in goods:
        goods[name] = []
        if unit:
            units[name] = unit

    goods[name].append({'amount': amount, 'expiration_date': expiration_date})


def add_by_note(note):
    """
    Добавляет продукт по заметке вида:
    '2.5 кг Пельмени Универсальные 2025-10-20'
    или '10 шт Яйцо'
    """
    parts = note.split()
    if len(parts) < 2:
        return

    amount_str = parts[0]
    unit = parts[1]  # например, 'кг', 'шт', 'л'
    amount = Decimal(amount_str)

    # Проверяем, есть ли дата в конце
    expiration_date = None
    last_part = parts[-1]
    if len(last_part) == 10 and last_part[4] == '-' and last_part[7] == '-':
        try:
            year = int(last_part[0:4])
            month = int(last_part[5:7])
            day = int(last_part[8:10])
            expiration_date = date(year, month, day)
            parts = parts[:-1]
        except ValueError:
            pass

    # Название — всё, что после количества и единицы
    product_name = ' '.join(parts[2:])

    # Сохраняю единицу измерения (если ещё не сохранена)
    if product_name not in units:
        units[product_name] = unit

    add(product_name, amount, expiration_date, unit)


def consume(name, amount_to_consume):
    """
    Уменьшает количество продукта (я что-то съел или выбросил)
    amount_to_consume — сколько съелось (в тех же единицах)
    """
    if name not in goods:
        print(f"Продукт '{name}' не найден")
        return

    # Преобразую в Decimal, но не перезаписываю имя функции 'amount'
    amount_to_consume = Decimal(str(amount_to_consume))
    total_available = amount(name)  # теперь amount — это снова функция

    if amount_to_consume > total_available:
        unit = units.get(name, '')
        print(f"Недостаточно продукта '{name}'. Доступно: {total_available} {unit}")
        return

    # Удаляю продукт из партий, начиная с самой первой (самой старой)
    remaining = amount_to_consume
    i = 0
    while remaining > 0 and i < len(goods[name]):
        batch = goods[name][i]
        if batch['amount'] <= remaining:
            # Вся партия уходит
            remaining -= batch['amount']
            goods[name].pop(i)
        else:
            # Частично использую партию
            batch['amount'] -= remaining
            remaining = 0
        # при pop индекс не увеличивается, так как список сдвинулся
    # Если продукт закончился — удаляем его полностью
    if not goods[name]:
        del goods[name]
        if name in units:
            del units[name]


def find(query):
    query = query.lower()
    return [name for name in goods if query in name.lower()]


def amount(name):
    if name not in goods:
        return Decimal('0')
    return sum(batch['amount'] for batch in goods[name])


def show_all():
    if not goods:
        print("Холодильник пуст")
        return

    for name in goods:
        unit = units.get(name, '')
        print(f"\n{name} ({unit}):")
        for i, batch in enumerate(goods[name], 1):
            exp = batch['expiration_date']
            exp_str = exp.strftime("%Y-%m-%d") if exp else "бессрочно"
            print(f"  Партия {i}: {batch['amount']} — годен до: {exp_str}")


def show_product(name):
    if name not in goods:
        print(f"Продукт '{name}' не найден в холодильнике")
        return

    unit = units.get(name, '')
    print(f"\n{name} ({unit}):")
    for i, batch in enumerate(goods[name], 1):
        exp = batch['expiration_date']
        exp_str = exp.strftime("%Y-%m-%d") if exp else "бессрочно"
        print(f"  Партия {i}: {batch['amount']} — годен до: {exp_str}")


# Пример использования
# Добавим продукты вручную
add("Молоко", 1.0, date(2025, 11, 1))
add("Хлеб", 1, date(2025, 10, 30))
add("Банан", 10, date(2025, 11, 10))

# Добавим по заметке
add_by_note("2.5 кг Пельмени Универсальные 2025-10-20")
add_by_note("1.5 л Вода")
add_by_note("5 шт Банан 2025-10-27")

# Показывает список всех имеющихся в холодильнике продуктов, и информацию о них
print("\n--- Весь холодильник ---")
show_all()

# Съедаю 7 бананов (по умолчанию из самой старой партии), ну или выкидываю их
consume("Банан", 7)

print("\n--- Информация о бананах после съедения---")
show_product("Банан")


--- Весь холодильник ---

Молоко ():
  Партия 1: 1.0 — годен до: 2025-11-01

Хлеб ():
  Партия 1: 1 — годен до: 2025-10-30

Банан (шт):
  Партия 1: 10 — годен до: 2025-11-10
  Партия 2: 5 — годен до: 2025-10-27

Пельмени Универсальные (кг):
  Партия 1: 2.5 — годен до: 2025-10-20

Вода (л):
  Партия 1: 1.5 — годен до: бессрочно

--- Информация о бананах после съедения---

Банан (шт):
  Партия 1: 3 — годен до: 2025-11-10
  Партия 2: 5 — годен до: 2025-10-27
